In [1]:
import numpy as np
import keras
import os
import pandas as pd
from keras import callbacks
import cv2
#from scipy import ndimage as ndi
import PIL
from PIL import Image
from sklearn.utils import shuffle
from os import listdir
from keras.models import Sequential,load_model
#from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras import backend as K
from keras.optimizers import Adam
#from keras.preprocessing import image
from sklearn.metrics import classification_report
#from keras import regularizers
#from skimage.color import rgb2gray
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

In [2]:
directory_root = "../input/plantdisease/plantvillage/PlantVillage/"
print(os.listdir("../input/plantdisease/plantvillage/PlantVillage/"))
print(len(listdir(directory_root)))

['Pepper__bell___Bacterial_spot', 'Potato___healthy', 'Tomato_Leaf_Mold', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato_Bacterial_spot', 'Tomato_Septoria_leaf_spot', 'Tomato_healthy', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato_Early_blight', 'Tomato__Target_Spot', 'Pepper__bell___healthy', 'Potato___Late_blight', 'Tomato_Late_blight', 'Potato___Early_blight', 'Tomato__Tomato_mosaic_virus']
15


In [3]:
image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    for plant_folder in root_dir :
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        print(f"Processing {plant_folder} ...")
      

        for image in plant_disease_folder_list:
            image_directory = f"{directory_root}/{plant_folder}/{image}"
            if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                image_list.append(image_directory)
                label_list.append(plant_folder)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

[INFO] Loading images ...
Processing Pepper__bell___Bacterial_spot ...
Processing Potato___healthy ...
Processing Tomato_Leaf_Mold ...
Processing Tomato__Tomato_YellowLeaf__Curl_Virus ...
Processing Tomato_Bacterial_spot ...
Processing Tomato_Septoria_leaf_spot ...
Processing Tomato_healthy ...
Processing Tomato_Spider_mites_Two_spotted_spider_mite ...
Processing Tomato_Early_blight ...
Processing Tomato__Target_Spot ...
Processing Pepper__bell___healthy ...
Processing Potato___Late_blight ...
Processing Tomato_Late_blight ...
Processing Potato___Early_blight ...
Processing Tomato__Tomato_mosaic_virus ...
[INFO] Image loading completed


In [4]:
img_data = pd.DataFrame({'ImagePath':image_list,'Label':label_list})
print(img_data.head())
print(len(img_data))

                                           ImagePath  \
0  ../input/plantdisease/plantvillage/PlantVillag...   
1  ../input/plantdisease/plantvillage/PlantVillag...   
2  ../input/plantdisease/plantvillage/PlantVillag...   
3  ../input/plantdisease/plantvillage/PlantVillag...   
4  ../input/plantdisease/plantvillage/PlantVillag...   

                           Label  
0  Pepper__bell___Bacterial_spot  
1  Pepper__bell___Bacterial_spot  
2  Pepper__bell___Bacterial_spot  
3  Pepper__bell___Bacterial_spot  
4  Pepper__bell___Bacterial_spot  
20636


In [5]:

img_data["LabelsInteger"] = None

index_labels_integer = img_data.columns.get_loc("LabelsInteger")

index_species = img_data.columns.get_loc("Label")

j = 0 
for i in range(len(img_data)):
    if i == 0:
        img_data.iloc[i, index_labels_integer] = j
    if i > 0:
        if img_data.iloc[i-1, index_species] == img_data.iloc[i, index_species]:
            img_data.iloc[i, index_labels_integer] = j
        else:
            j += 1
            img_data.iloc[i, index_labels_integer] = j
#img_data[img_data["label"].unique()]
#print(np.unique(img_data['labels_integer']))

In [7]:
#crosschecking interger label assigned properly or not 
img_data.drop_duplicates(subset = ["LabelsInteger"])

,ImagePath,Label,LabelsInteger
0,../input/plantdisease/plantvillage/PlantVillag...,Pepper__bell___Bacterial_spot,0
997,../input/plantdisease/plantvillage/PlantVillag...,Potato___healthy,1
1149,../input/plantdisease/plantvillage/PlantVillag...,Tomato_Leaf_Mold,2
2101,../input/plantdisease/plantvillage/PlantVillag...,Tomato__Tomato_YellowLeaf__Curl_Virus,3
5309,../input/plantdisease/plantvillage/PlantVillag...,Tomato_Bacterial_spot,4
7436,../input/plantdisease/plantvillage/PlantVillag...,Tomato_Septoria_leaf_spot,5
9207,../input/plantdisease/plantvillage/PlantVillag...,Tomato_healthy,6
10798,../input/plantdisease/plantvillage/PlantVillag...,Tomato_Spider_mites_Two_spotted_spider_mite,7
12474,../input/plantdisease/plantvillage/PlantVillag...,Tomato_Early_blight,8
13474,../input/plantdisease/plantvillage/PlantVillag...,Tomato__Target_Spot,9


In [9]:
img_data= shuffle(img_data)
list_vectors = []

for image_path in img_data.ImagePath:
   
    img = Image.open(image_path)
    size = (64, 64)
    img = img.resize(size, PIL.Image.ANTIALIAS)  
    img_array = np.array(img)
    list_vectors.append(img_array)
    
print(len(list_vectors))

20636


In [10]:
X = np.stack((list_vectors))
Y =  img_data['LabelsInteger']
print(X.shape)

(20636, 64, 64, 3)


In [11]:
X = X/255
Y_one_hot = keras.utils.to_categorical(Y, num_classes=15)
print(Y_one_hot[0:15])
print(Y.shape, Y_one_hot.shape)

[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
(20636,) (20636, 15)


In [12]:
np.savez("x_images_arrayscnn", X)
np.savez("y_numeric_labelscnn", Y_one_hot)

In [13]:
x_npz = np.load("x_images_arrayscnn.npz")
X = x_npz['arr_0']

y_npz = np.load("y_numeric_labelscnn.npz")
Y_one_hot = y_npz['arr_0']

print(X.shape)

(20636, 64, 64, 3)


In [14]:
split_train = 0.8 
split_val = 0.9
index_train = int(split_train*len(X))
index_val = int(split_val*len(X))

X_train = X[:index_train]
X_val = X[index_train:index_val]
X_test = X[index_val:]

Y_train = Y_one_hot[:index_train]
Y_val = Y_one_hot[index_train:index_val]
Y_test = Y_one_hot[index_val:]


print(X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape)

(16508, 64, 64, 3) (2064, 64, 64, 3) (2064, 64, 64, 3) (16508, 15) (2064, 15) (2064, 15)


In [15]:
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3]) 
num_classes = 15

model = Sequential()
model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(64, (5, 5)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(1000))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


model.compile(optimizer=keras.optimizers.Adam(lr=0.0002, decay=1e-7),loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
 best_model_file = "Leaf_disease.h5"
best_model = ModelCheckpoint(best_model_file, monitor='val_loss', verbose=1, save_best_only=True)

print('Training model')
results = model.fit(X_train, Y_train, epochs=50, batch_size=64, validation_data=(X_val, Y_val), callbacks=[best_model])
print('Traing finished')


In [ ]:
scores=model.evaluate(X_test, Y_test)
print(f"Test Accuracy: {scores[1]*100}")

In [ ]:
Y_test = np.argmax(Y_test,axis=1)
y_pred = model.predict_classes(X_test)
print(classification_report(Y_test, y_pred))

In [ ]:
acc = results.history['accuracy']
val_acc = results.history['val_accuracy']
loss = results.history['loss']
val_loss = results.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()

plt.show()